# scrypt (Password Hashing)

scrypt is a password hashing function designed by Colin Percival in 2009. It was created to be memory-hard, which forces attackers using GPUs or ASICs to expend a lot of RAM during a brute-force attack, thereby making such attacks significantly slower and more expensive.

---

## Key Features

- **Memory-Hard:**  
  scrypt requires significant memory, which helps mitigate attacks from massively parallel hardware.

- **Configurable Parameters:**  
  - **N (Cost Factor):** A power of 2 (e.g., 2^14 = 16384) that defines the CPU and memory cost.
  - **r (Block Size):** Usually set to 8.
  - **p (Parallelism):** The number of parallel computational threads (e.g., 1).
  - **dkLen (Derived Key Length):** The length (in bytes) of the output hash (e.g., 32 bytes).

- **Built-In Salt Generation:**  
  A unique, random salt (typically 16+ bytes) is used to thwart precomputed rainbow table attacks.

- **Security Tradeoffs:**  
  While scrypt is stronger than bcrypt against GPU/ASIC attacks due to its memory hardness, the newer Argon2id is generally recommended for even stronger security.

---

## How scrypt Works

1. **Input:**  
   The function takes a user's password and a unique salt.

2. **PBKDF2 Preprocessing:**  
   It uses PBKDF2 with HMAC-SHA-256 to generate initial key material.

3. **Memory-Hard Mixing:**  
   The algorithm fills a large memory array with pseudorandom data and repeatedly mixes it. This step is deliberately resource-intensive.

4. **Compression:**  
   The mixed data is then compressed to produce the final derived key (hash).

---

## Communication Flow (Registration and Login Scenario)

1. **Registration:**
   - **User Input:**  
     Alice chooses a password, e.g., `"mypassword"`.
   - **Hashing:**  
     scrypt generates a hash using the password, a random salt, and configured parameters.
   - **Storage:**  
     The resultant hash (with embedded parameters and salt) is stored in the database.

2. **Login:**
   - **Password Submission:**  
     Alice inputs her password.
   - **Verification:**  
     The system re-computes the scrypt hash with the stored salt and parameters.
   - **Matching:**  
     If the computed hash matches the stored value, access is granted; otherwise, it is denied.

3. **Attack Resistance:**  
   Even if an attacker steals the database, the memory-heavy computation makes brute-forcing scrypt hashes extremely resource-intensive.

---

## Parameters Summary

- **Password:** User-provided input.
- **Salt:** A random, unique salt (recommended 16+ bytes).
- **N (Cost Factor):** Specifies CPU/memory cost (e.g., 2^14 = 16384).
- **r (Block Size):** Typically 8.
- **p (Parallelism):** Number of parallel threads, such as 1.
- **dkLen:** Desired length of the derived key in bytes (e.g., 32).

---

## Security Notes

- **Stronger than bcrypt:**  
  scrypt’s memory-hard design offers improved resistance against brute-force attacks using specialized hardware.

- **Parameter Sensitivity:**  
  Incorrect parameter choices (too low N, r, or p) can weaken security. Choose values that balance security with acceptable performance.

- **Current Best Practice:**  
  Though scrypt remains secure when properly configured, Argon2id is generally recommended for new applications due to its improved design and resistance to side-channel attacks.

---

## Real-World Applications

- **Cryptocurrency Wallets:**  
  Used for key derivation in some wallets (e.g., Litecoin, Dogecoin).

- **Secure Password Storage:**  
  Adopted by various libraries and systems for storing hashed passwords.

- **Key Derivation for File Encryption:**  
  Applied in scenarios where a derived key is used to encrypt files.

---

In [1]:
import hashlib, os

# --- Parameters ---
password = b"SuperSecret123!"
salt = os.urandom(16)      # random 16-byte salt
N = 2**14                  # CPU/memory cost
r = 8                      # block size
p = 1                      # parallelism
dklen = 32                 # output key length

# --- Hash password ---
hash_bytes = hashlib.scrypt(password, salt=salt, n=N, r=r, p=p, dklen=dklen)

print("Salt (hex):", salt.hex())
print("Derived key (hex):", hash_bytes.hex())

# --- Verifying (rehash with same salt + params) ---
check = hashlib.scrypt(password, salt=salt, n=N, r=r, p=p, dklen=dklen)
print("Password correct?", check == hash_bytes)


Salt (hex): bc7090c9208b6dff3efc2aeb1881bd66
Derived key (hex): 0b3ff9f9109fb9205b44952749838c80f1b770482e227405cdebb94711975968
Password correct? True
